<font size = "20"> Spotify Music Analysis </font>

In this project, we will attempt to analyze Spotify playlists to see how compatible two user's playlists are. We will also use content-based filtering to create a recommendation system. 

In [107]:
# All imports are listed here
import requests
import spotipy.util as util
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from json.decoder import JSONDecodeError

The first step in the process is to authenticate with the Spotify API and then import both of our playlists. The songs of both playlists are printed below

In [114]:
# Authentication with Spotify API
sp = spotipy.Spotify()
cid ="9eab3f44cb5f42bd9d144ff2e2ad767f"
secret = "677567e07a1c4d9980adc0da6868b316"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False # What is this?

# This is Shubham's playlist
playlist = sp.user_playlist("1232443238", "5XpKz5MtjD6Vxh8mN3fhK7")
songs = playlist["tracks"]["items"]
ids = []




In [109]:
# Printing out playlist name and total tracks
print('Playlist Name:', playlist['name'], '(Shubhams\' Playlist)')
print('Total Tracks:', playlist['tracks']['total'])

# Obtain tracks from Shubham's playlist
results = sp.user_playlist("1232443238", playlist['id'], fields="tracks,next")
tracks = results['tracks']

# Print out all tracks from Shubham's playlist using a counter
counter = 1
for i in results['tracks']['items']:
    #print(counter, i['track']['artists'][0]['name'] + ' - ' + i['track']['name'])
    counter = counter + 1

Playlist Name: Epic Songs (Shubhams' Playlist)
Total Tracks: 45


For the next step, we want to find out what the feature values are for each song in every playlist. Before the values are manipluated, for both playlists we will first normalize all of the numerical values and then find the average value for the entire playlist

In [118]:
# Prints out all feature values in my playlist
for i in range(len(songs)):
    ids.append(songs[i]["track"]["id"])
features = sp.audio_features(ids)

# Converting to dataframe and print
df = pd.DataFrame(features)
#df


In [121]:
# Normalizing step for the 13 numeric columns? (Shubham's playlist)
norm = df.copy()
music1mean = []
for col in df:
    if (df[col].dtype != 'object'):
        norm[col] = norm[col] /norm[col].abs().max()
        music1mean.append(norm[col].mean())
        
#music1mean
